!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "tr", split="test", use_auth_token=True)

print(common_voice)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [4]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [6]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Turkish", task="transcribe")

In [8]:
print(common_voice["train"][0])

In [9]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [10]:
print(common_voice["train"][0])

In [11]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [13]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"],num_proc=8)

Map (num_proc=8):   0%|          | 0/36125 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/10143 [00:00<?, ? examples/s]

In [15]:
help(common_voice)

In [22]:
import os

['dev',
 'etc',
 'local',
 'proc',
 'run',
 'sys',
 'tmp',
 'var',
 'usr',
 'bin',
 'sbin',
 'lib',
 'lib64',
 'boot',
 'home',
 'media',
 'mnt',
 'opt',
 'root',
 'srv',
 '.autorelabel']

In [23]:
common_voice.save_to_disk("dataset")

Saving the dataset (0/70 shards):   0%|          | 0/36125 [00:00<?, ? examples/s]

Saving the dataset (0/20 shards):   0%|          | 0/10143 [00:00<?, ? examples/s]

In [26]:
common_voice.save_to_disk("s3://project-whisper-data/dataset")

Saving the dataset (0/70 shards):   0%|          | 0/36125 [00:00<?, ? examples/s]

Saving the dataset (0/20 shards):   0%|          | 0/10143 [00:00<?, ? examples/s]